In [1]:
# Essential installation.
!pip install yt-dlp
!pip install pinecone-client
!pip install langchain
!pip install langchain_openai
!pip install openai
!pip install wikipedia-api
!pip install langchain_community
!pip install transformers
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 26.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.2 MB/s eta 0:00:0

In [2]:
# Define API keys.
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
HF_TOKEN = userdata.get('HF_TOKEN')
LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
YOUTUBE_API_KEY = userdata.get('YOUTUBE_API_KEY')

In [3]:
# Import necessary libraries.
import os
import re
import json
from tqdm import tqdm
from urllib.parse import urlparse, parse_qs
from googleapiclient.discovery import build
import yt_dlp
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import pinecone
import wikipediaapi
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
import torch

In [4]:
def clean_text(text, for_filename=False):
    # Replace newlines and carriage returns with spaces.
    text = text.replace('\n', ' ').replace('\r', ' ')

    if for_filename:
        # Remove special characters for file names.
        text = ''.join(e for e in text if e.isalnum() or e.isspace())
    else:
        # Replace non-alphanumeric characters (except spaces) with spaces.
        text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

    # Normalize multiple spaces into a single space.
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def extract_video_id(url):
    query = urlparse(url).query
    params = parse_qs(query)
    return params.get('v', [None])[0]

def extract_playlist_id(url):
    query = urlparse(url).query
    params = parse_qs(query)
    return params.get('list', [None])[0]

In [5]:
# Metadata search and save.
def save_metadata(metadata_list, metadata_path):
    if not os.path.exists(metadata_path):
        os.makedirs(metadata_path)

    all_metadata_filename = os.path.join(metadata_path, 'all_metadata.json')
    with open(all_metadata_filename, 'w') as all_metadata_file:
        json.dump(metadata_list, all_metadata_file, indent=4)

def fetch_video_metadata(video_id, api_key, metadata_list):
    youtube = build('youtube', 'v3', developerKey=api_key)
    request = youtube.videos().list(
        part="snippet",
        id=video_id
    )
    response = request.execute()

    if not response['items']:
        print(f"No metadata found for video ID: {video_id}")
        return None

    item = response['items'][0]
    video_metadata = {
        'video_id': video_id,
        'title': clean_text(item['snippet']['title']),
        'description': clean_text(item['snippet']['description']),
        'published_at': item['snippet']['publishedAt']
    }

    metadata_list.append(video_metadata)
    return video_metadata

def fetch_playlist_metadata(playlist_id, api_key, metadata_list):
    youtube = build('youtube', 'v3', developerKey=api_key)
    request = youtube.playlistItems().list(
        part="snippet",
        playlistId=playlist_id,
        maxResults=50
    )

    while request:
        response = request.execute()
        for item in response['items']:
            video_id = item['snippet']['resourceId']['videoId']
            fetch_video_metadata(video_id, api_key, metadata_list)

        request = youtube.playlistItems().list_next(request, response)
    return metadata_list

Cell 3: YouTube Download Function

In [6]:
def download_from_youtube(download_path, metadata_path, api_key):
    video_url = "https://www.youtube.com/watch?v=GFLb5h2O2Ww&list=PLFs4vir_WsTyS2cy4vj4obl5igqCOV749"
    playlist_id = extract_playlist_id(video_url)
    metadata_list = []

    print("Fetching metadata...")
    fetch_playlist_metadata(playlist_id, api_key, metadata_list)
    save_metadata(metadata_list, metadata_path)
    print("Metadata saved successfully.")

    if not os.path.exists(download_path):
        os.makedirs(download_path)

    print("Downloading playlist from YouTube...")

    pbar = None

    def tqdm_hook(d):
        nonlocal pbar
        if d['status'] == 'downloading':
            if pbar is None:
                pbar = tqdm(total=d['total_bytes'], unit='B', unit_scale=True, unit_divisor=1024)
            pbar.update(d['downloaded_bytes'] - pbar.n)
        elif d['status'] == 'finished':
            if pbar:
                pbar.close()

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{download_path}/%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'verbose': True,
        'progress_hooks': [tqdm_hook]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    print("Playlist downloaded successfully.")

    return metadata_list

metadata_path = './metadata'
download_path = './downloads'
pre_fetched_metadata = download_from_youtube(download_path, metadata_path, YOUTUBE_API_KEY)

Fetching metadata...


[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8 (No ANSI), error UTF-8 (No ANSI), screen UTF-8 (No ANSI)
[debug] yt-dlp version stable@2024.07.16 from yt-dlp/yt-dlp [89a161e8c] (pip) API
[debug] params: {'format': 'bestaudio/best', 'outtmpl': './downloads/%(title)s.%(ext)s', 'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '192'}], 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0', 'verbose': True, 'progress_hooks': [<function download_from_youtube.<locals>.tqdm_hook at 0x7eed658f6b00>], 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}}
[debug] Python 3.10.12 (CPython x86_64 64bit) - Linux-6.1.85+-x86_64-with-glibc2.35 

Metadata saved successfully.


[debug] exe versions: ffmpeg 4.4.2 (setts), ffprobe 4.4.2
[debug] Optional libraries: Cryptodome-3.20.0, brotli-1.1.0, certifi-2024.07.04, mutagen-1.47.0, requests-2.32.3, secretstorage-3.3.1, sqlite3-3.37.2, urllib3-2.0.7, websockets-12.0
[debug] Proxy map: {'colab_language_server': '/usr/colab/bin/language_service'}
[debug] Request Handlers: urllib, requests, websockets
[debug] Loaded 1829 extractors


[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=GFLb5h2O2Ww&list=PLFs4vir_WsTyS2cy4vj4obl5igqCOV749
[youtube:tab] Downloading playlist PLFs4vir_WsTyS2cy4vj4obl5igqCOV749 - add --no-playlist to download just the video GFLb5h2O2Ww
[youtube:tab] PLFs4vir_WsTyS2cy4vj4obl5igqCOV749: Downloading webpage
[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLFs4vir_WsTyS2cy4vj4obl5igqCOV749
[youtube:tab] PLFs4vir_WsTyS2cy4vj4obl5igqCOV749: Downloading webpage
[youtube:tab] PLFs4vir_WsTyS2cy4vj4obl5igqCOV749: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Immune System
[youtube:tab] PLFs4vir_WsTyS2cy4vj4obl5igqCOV749 page 1: Downloading API JSON


...

[download] Destination: ./downloads/The Immune System Explained I – Bacteria Infection.webm
[download] 100% of    6.86MiB in 00:00:00 at 11.35MiB/s  


[debug] ffmpeg command line: ffprobe -show_streams 'file:./downloads/The Immune System Explained I – Bacteria Infection.webm'


[ExtractAudio] Destination: ./downloads/The Immune System Explained I – Bacteria Infection.mp3


[debug] ffmpeg command line: ffmpeg -y -loglevel repeat+info -i 'file:./downloads/The Immune System Explained I – Bacteria Infection.webm' -vn -acodec libmp3lame -b:a 192.0k -movflags +faststart 'file:./downloads/The Immune System Explained I – Bacteria Infection.mp3'


Deleting original file ./downloads/The Immune System Explained I – Bacteria Infection.webm (pass -k to keep)
[download] Finished downloading playlist: Immune System
Playlist downloaded successfully.


Cell 4: Transcription Function

In [7]:
def transcribe_audio_hf(mp3_path, transcript_path):
    if not os.path.exists(transcript_path):
        os.makedirs(transcript_path)
    mp3_files = [f for f in os.listdir(mp3_path) if f.endswith('.mp3')]

    if not mp3_files:
        print("No MP3 files found for transcription.")
        return

    processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3")
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")
    feature_extractor = processor.feature_extractor

    # Dynamically select the device (handy for deployment environment).
    device = 0 if torch.cuda.is_available() else -1
    transcriber = pipeline("automatic-speech-recognition", model=model, tokenizer=processor.tokenizer, feature_extractor=feature_extractor, chunk_length_s=30, device=device)

    transcripts = []

    for mp3_file in tqdm(mp3_files, desc="Transcribing audio"):
        audio_path = os.path.join(mp3_path, mp3_file)
        print(f"Transcribing {mp3_file} located at {audio_path}...")

        try:
            # Generate transcription.
            transcript = transcriber(audio_path)
            transcript_text = transcript['text']
            cleaned_transcript = clean_text(transcript_text)

            # Add title (filename) and transcript to the transcripts list.
            transcripts.append({
                'title': os.path.splitext(mp3_file)[0],
                'transcript': cleaned_transcript
            })

            print(f"Transcription for {mp3_file} completed.")

        except Exception as e:
            print(f"Error transcribing {mp3_file}: {e}")

    # Save all transcripts to a single JSON file.
    all_transcripts_filename = os.path.join(transcript_path, 'all_transcripts.json')
    with open(all_transcripts_filename, 'w') as all_transcripts_file:
        json.dump(transcripts, all_transcripts_file, indent=4)

    print(f"All transcripts saved to {all_transcripts_filename}")

transcript_path = './transcripts'
transcribe_audio_hf(download_path, transcript_path)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

Transcription for The Reason Why Cancer is so Hard to Beat.mp3 completed.
Transcribing Why Blue Whales Don't Get Cancer - Peto's Paradox.mp3 located at ./downloads/Why Blue Whales Don't Get Cancer - Peto's Paradox.mp3...


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Transcribing audio:  95%|█████████▌| 19/20 [32:06<01:28, 88.04s/it]

Transcription for Why Blue Whales Don't Get Cancer - Peto's Paradox.mp3 completed.
Transcribing The Ebola Virus Explained — How Your Body Fights For Survival.mp3 located at ./downloads/The Ebola Virus Explained — How Your Body Fights For Survival.mp3...


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Transcribing audio: 100%|██████████| 20/20 [33:01<00:00, 99.08s/it]

Transcription for The Ebola Virus Explained — How Your Body Fights For Survival.mp3 completed.
All transcripts saved to ./transcripts/all_transcripts.json


Cell 5: Preprocessing and Embedding Function

In [8]:
def initialize_pinecone(api_key, index_name, dimension=1536):
    from pinecone import Pinecone, ServerlessSpec

    # Create an instance of the Pinecone class
    pc = Pinecone(api_key=api_key)

    # Check if the index already exists
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

    index = pc.Index(index_name)
    return index

pinecone_api_key = PINECONE_API_KEY
index_name = 'youtube-videos-data'

# Initialize Pinecone
index = initialize_pinecone(pinecone_api_key, index_name)

print("Pinecone index initialized successfully.")

Pinecone index initialized successfully.


In [9]:
def preprocess_and_embed(transcript_path, langchain_api_key):
    # Initialize the text splitter.
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=150)

    # Collect all transcripts.
    transcripts = []
    all_transcripts_filename = os.path.join(transcript_path, 'all_transcripts.json')
    if os.path.exists(all_transcripts_filename):
        with open(all_transcripts_filename, 'r') as all_transcripts_file:
            transcripts_data = json.load(all_transcripts_file)
            for item in transcripts_data:
                transcripts.append(item['transcript'])

    # Split and tokenize transcripts.
    text_chunks = []
    for transcript in transcripts:
        text_chunks.extend(text_splitter.split_text(transcript))

    embeddings_model = OpenAIEmbeddings(api_key=langchain_api_key)

    # Generate embeddings for each chunk.
    embeddings = embeddings_model.embed_documents(text_chunks)

    assert len(text_chunks) == len(embeddings), "Number of text chunks and embeddings do not match."

    return text_chunks, embeddings

# Test function.

langchain_api_key = OPENAI_API_KEY
# Call the function and check the output.
text_chunks, embeddings = preprocess_and_embed(transcript_path, langchain_api_key)

print(f"Number of text chunks generated: {len(text_chunks)}")

print("Text Chunks:")
for chunk in text_chunks[:5]:
    print(chunk)
    print()

print("Embeddings:")
for embedding in embeddings[:5]:
    print(embedding)
    print()

Number of text chunks generated: 205
Text Chunks:
Few of the monsters that evolution created have been so successful at hurting us as the variola virus responsible for smallpox The carnage it caused was so terrible and merciless that it compelled humankind for the first time to act truly globally It was one of the greatest wins of our species over the ancient powers of nature all made possible by cows Variola is a virus a tiny machine that only seeks to reproduce itself Evidence of it has been found in Egyptian mummies and in writing from India and China as old as 3 000 years 1 300 years ago smallpox killed up to a third of Japan s population By the 1600s it was one of the major causes of death worldwide In late 18th century Europe it killed 400 000 a year Every third person who went blind did so because of this virus Even in the 20th century a hot second ago in history it still killed at least 300 million people Smallpox is an abusive monster that returns over and over and over again 

In [10]:
def upsert_embeddings(index, text_chunks, embeddings):
    # Upsert embeddings into Pinecone index.
    for i, (chunk, embedding) in enumerate(zip(text_chunks, embeddings)):
        index.upsert([(str(i), embedding, {'text': chunk})])

# Upsert embeddings.
upsert_embeddings(index, text_chunks, embeddings)

print("Embeddings upserted into Pinecone index.")

Embeddings upserted into Pinecone index.


In [11]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 208}},
 'total_vector_count': 208}

In [13]:
!pip install -U langchain-openai
!pip install -U langchain-pinecone


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 5.0.0
    Uninstalling pinecone-client-5.0.0:
      Successfully uninstalled pinecone-client-5.0.0


In [14]:
from langchain_openai import OpenAIEmbeddings  # Updated import
from langchain_pinecone import Pinecone          # Updated import

text_field = 'text'
# Initialize the embeddings model
langchain_api_key = OPENAI_API_KEY
openai_embeddings = OpenAIEmbeddings(api_key=langchain_api_key)

# Initialize the vector store object
vectorstore = Pinecone(
    index, openai_embeddings, text_field  # Pass the instance directly
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.3 and will be removed in 0.3.0. Use PineconeVectorStore instead.
  warn_deprecated(


In [15]:
query = "what is immune system??"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='whiny little baby your immune system is not complaining It is busy keeping you alive so you can live to whine another day And so while intruders roam your body killing hundreds of thousands of your cells your immune system is organizing complex defenses communicating over vast distances activating intricate defense networks and dishing out a swift death to millions if not billions of enemies all while you are standing in the shower mildly annoyed But this complexity is largely hidden Which is a real shame because there are not many things that have such a crucial impact on the quality of your life as your immune system It is all embracing and all encompassing protecting you from bothersome nuisances like the common cold scratches and cuts to life threatening stuff from cancer and pneumonia to deadly infections like COVID 19 Your immune system is as indispensable as your heart or your lungs And actually it is one of the largest and most widespread organ systems t

In [43]:
all_transcripts = './transcripts'

# Function to read transcripts from a single JSON file
def read_transcripts(transcript_file):
    # Construct the full path to the transcript file
    file_path = os.path.join(all_transcripts, transcript_file) # Use os.path.join to create the correct file path
    with open(file_path, 'r') as file:
        transcripts = json.load(file)
    return transcripts
'''
# Function to summarize text
def get_summary(transcript_file):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    transcripts = read_transcripts(transcript_file)

    summaries = []
    for transcript in transcripts:
        summary = summarizer(transcript['transcript'], max_length=150, min_length=30, do_sample=False)[0]['summary_text']
        summaries.append(summary)

    return summaries


summarizer_tool = Tool(
    name="Summarizer",
    func=lambda x: get_summary(transcript_file=x), # Pass the filename to get_summary
    description="Summarize the content of transcripts. Find summaries based on user questions."
)
'''
# Initialize memory.
memory = ConversationBufferMemory()

# Initialize the language model using OpenAI with the specific model.
llm = OpenAI(api_key=OPENAI_API_KEY, temperature=0.7)

from langchain.chains import RetrievalQA
# Create the RetrievalQA instance with the language model and retriever
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory  # Integrate the memory with RetrievalQA
)

# Define tools for the agent
tools = [
    Tool(
        name='video_transcript_retriever',
        func=qa.run,
        description='Searches and returns excerpts from the transcript of the user-uploaded video.'
    )
]

# Initialize the agent
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, memory=memory, verbose=True
)

print("QA Agent initialized successfully.")

QA Agent initialized successfully.


In [54]:
query1 = "What are the potential negative impacts of getting a tattoo on the immune system?"


output1 = agent.run(query1)
print(output1)



> Entering new AgentExecutor chain...
 It would be helpful to gather information from a reliable source.
Action: video_transcript_retriever
Action Input: search terms "tattoo immune system negative impacts"
Observation:  Some possible results for this search could include articles or studies discussing potential negative impacts on the immune system from getting a tattoo, such as increased risk of infection or allergic reactions to ink components. It may also include information on how the immune system responds to the tattooing process and the presence of foreign substances in the body.
Thought: It would also be beneficial to consider the potential positive impacts of getting a tattoo on the immune system, as well as any mitigating factors that could lessen the negative impacts.
Action: video_transcript_retriever
Action Input: search terms "tattoo immune system positive impacts"
Observation:  The positive impacts of tattoos on the immune system include the activation of macrophages 

In [55]:
query2 = "What is Ebola virus?"


output2 = qa.invoke(query2)
output2['result']

' Ebola virus is a highly dangerous and deadly virus that attacks the immune system and causes severe illness and often death. It is transmitted through contact with bodily fluids of infected individuals or animals.'

In [56]:
query3 = "Does the human body kill canser and how?"


output3 = qa.invoke(query3)
output3['result']

' Yes, the human body has mechanisms in place to detect and kill cancer cells. These mechanisms include the immune system, which can recognize and attack cancer cells, as well as cell suicide (apoptosis) which allows abnormal or damaged cells to self-destruct. Additionally, the body has repair mechanisms to fix mutations in DNA that can lead to cancer.'

In [57]:
query4 = "Are the human immune against every disease?"


output4 = qa.invoke(query4)
output4['result']

' No, humans are not immune against every disease. Our immune system can protect us against many diseases, but there are still many diseases that can affect us and even cause serious illness or death.'

In [58]:
query5 = "Why Cancer is so Hard to Beat?"


output5 = qa.invoke(query5)
output5['result']

" Cancer is difficult to beat because it is constantly evolving and adapting to treatments, making it difficult to eradicate completely. Additionally, the body's immune system can sometimes accidentally select and strengthen the fittest tumor cells, allowing them to continue growing and spreading."

In [74]:
!pip install -U langsmith

Evaluation:

In [79]:
#from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain.evaluation import load_evaluator


evaluator = load_evaluator("labeled_criteria", llm=llm, criteria="correctness")

In [80]:
from langsmith.evaluation import evaluate

evaluator = load_evaluator("labeled_criteria", llm=llm, criteria="conciseness")
evaluator = load_evaluator("labeled_criteria", llm=llm, criteria="coherence")

In [83]:
# Verify the type and content of output1
print(type(output1))
print(output1)

# If output1 is a string and contains a JSON-like structure, parse it
import json
try:
    output1_dict = json.loads(output1)
    val_result = evaluator.evaluate_strings(
        prediction=output1_dict['result'],
        input=query1,
        reference=""
    )
    print(val_result)
except json.JSONDecodeError:
    print("output1 is not a valid JSON string.")

<class 'str'>
The potential negative impacts of getting a tattoo on the immune system include increased risk of infection and allergic reactions, while the potential positive impacts include activation of immune cells and trapping of harmful substances. It is important to consider these factors before making a decision about getting a tattoo.
output1 is not a valid JSON string.


In [95]:
print(type(output2))
print(output2)

output2_dict = output2
val_result = evaluator.evaluate_strings(
    prediction=output2_dict['result'],
    input=query2,
    reference=""
)
print(val_result)

<class 'dict'>
{'query': 'What is Ebola virus?', 'history': 'Human: Why Cancer is so Hard to Beat?\nAI:  Cancer is hard to beat because it is a complex disease that constantly evolves and mutates, making it difficult for treatments to target and eradicate all the cells. Additionally, the body\'s natural defense mechanisms can accidentally select for the strongest and most resilient cancer cells, allowing them to survive and continue growing. Scientists are constantly researching and developing new treatments, but it is a difficult and ongoing battle.\nHuman: search terms "tattoo immune system negative impacts\nAI:  Some possible results for this search could include articles or studies discussing potential negative impacts on the immune system from getting a tattoo, such as increased risk of infection or allergic reactions to ink components. It may also include information on how the immune system responds to the tattooing process and the presence of foreign substances in the body.\nHu

In [97]:
print(type(output3))
print(output3)

output3_dict = output3
val_result = evaluator.evaluate_strings(
    prediction=output3_dict['result'],
    input=query3,
    reference=""
)
print(val_result)

<class 'dict'>
{'query': 'Does the human body kill canser and how?', 'history': 'Human: Why Cancer is so Hard to Beat?\nAI:  Cancer is hard to beat because it is a complex disease that constantly evolves and mutates, making it difficult for treatments to target and eradicate all the cells. Additionally, the body\'s natural defense mechanisms can accidentally select for the strongest and most resilient cancer cells, allowing them to survive and continue growing. Scientists are constantly researching and developing new treatments, but it is a difficult and ongoing battle.\nHuman: search terms "tattoo immune system negative impacts\nAI:  Some possible results for this search could include articles or studies discussing potential negative impacts on the immune system from getting a tattoo, such as increased risk of infection or allergic reactions to ink components. It may also include information on how the immune system responds to the tattooing process and the presence of foreign substan

In [98]:
print(type(output4))
print(output4)

output4_dict = output4
val_result = evaluator.evaluate_strings(
    prediction=output4_dict['result'],
    input=query4,
    reference=""
)
print(val_result)

<class 'dict'>
{'query': 'Are the human immune against every disease?', 'history': 'Human: Why Cancer is so Hard to Beat?\nAI:  Cancer is hard to beat because it is a complex disease that constantly evolves and mutates, making it difficult for treatments to target and eradicate all the cells. Additionally, the body\'s natural defense mechanisms can accidentally select for the strongest and most resilient cancer cells, allowing them to survive and continue growing. Scientists are constantly researching and developing new treatments, but it is a difficult and ongoing battle.\nHuman: search terms "tattoo immune system negative impacts\nAI:  Some possible results for this search could include articles or studies discussing potential negative impacts on the immune system from getting a tattoo, such as increased risk of infection or allergic reactions to ink components. It may also include information on how the immune system responds to the tattooing process and the presence of foreign subs

In [99]:
print(type(output5))
print(output5)

output5_dict = output5
val_result = evaluator.evaluate_strings(
    prediction=output5_dict['result'],
    input=query5,
    reference=""
)
print(val_result)

<class 'dict'>
{'query': 'Why Cancer is so Hard to Beat?', 'history': 'Human: Why Cancer is so Hard to Beat?\nAI:  Cancer is hard to beat because it is a complex disease that constantly evolves and mutates, making it difficult for treatments to target and eradicate all the cells. Additionally, the body\'s natural defense mechanisms can accidentally select for the strongest and most resilient cancer cells, allowing them to survive and continue growing. Scientists are constantly researching and developing new treatments, but it is a difficult and ongoing battle.\nHuman: search terms "tattoo immune system negative impacts\nAI:  Some possible results for this search could include articles or studies discussing potential negative impacts on the immune system from getting a tattoo, such as increased risk of infection or allergic reactions to ink components. It may also include information on how the immune system responds to the tattooing process and the presence of foreign substances in the